In [1]:
include("src/sgs_store.jl")

using ..Entity
using ..HllSets

using PyCall
using CSV
using DataFrames
using WordTokenizers

redis   = pyimport("redis")
# Connect to Redis
r = redis.Redis(host="localhost", port=6379, db=0)

csv_file_path = "/home/alexmy/Downloads/POC/DATA/enron_05_17_2015_with_labels_v2.csv"

println(typeof(r))


PyObject


In [2]:
df      = DataFrame(CSV.File(csv_file_path, header=true))
df.From = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.From)
df.To   = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.To)

df_filled = coalesce.(df, "unknown")

print()

In [3]:
"""
    1. Message-ID	
    2. Date	
    3. From	
    4. To	
    5. Subject	
    6. X-From	X-To	X-cc	X-bcc	X-Folder	X-Origin	X-FileName	
    13. content	
    14. user	
    15. Cat_1_level_1	Cat_1_level_2	Cat_1_weight	Cat_2_level_1	Cat_2_level_2	Cat_2_weight	Cat_3_level_1	Cat_3_level_2	Cat_3_weight	Cat_4_level_1	Cat_4_level_2	Cat_4_weight	Cat_5_level_1	Cat_5_level_2	Cat_5_weight	Cat_6_level_1	Cat_6_level_2	Cat_6_weight	Cat_7_level_1	Cat_7_level_2	Cat_7_weight	Cat_8_level_1	Cat_8_level_2	Cat_8_weight	Cat_9_level_1	Cat_9_level_2	Cat_9_weight	Cat_10_level_1	Cat_10_level_2	Cat_10_weight	Cat_11_level_1	Cat_11_level_2	Cat_11_weight	Cat_12_level_1	Cat_12_level_2	Cat_12_weight	
    51. labeled
"""

# Initialize sets to collect data from specific columns
hll_03 = HllSets.HllSet{10}()     # Set{String}()
hll_04 = HllSets.HllSet{10}()     # Set{String}()
hll_05 = HllSets.HllSet{10}()     # Set{String}()
hll_06 = HllSets.HllSet{10}()     # Set{String}()
hll_14 = HllSets.HllSet{10}()     # Set{String}()
hll_15 = HllSets.HllSet{10}()     # Set{String}()


HllSet{10}()


In [4]:
using ..Store
using ..Util
using JSON3
using TextAnalysis
using WordTokenizers
using Base.Threads

tokenizer = WordTokenizers.Words
# r::PyObject, df::DataFrame, parent::String, cols::Vector; p::Int=10, chunk_size::Int=512000
# Store.ingest_df(r, tokenizer, df, csv_file_path, [:From, :To, :Subject, :content, :user])

cols = [:From, :To, :Subject, :content, :user]
p::Int=10 
chunk_size::Int=512000
parent = csv_file_path

for column in cols    
    col_values  = df[:, column]
    col_sha1    = Util.sha1_union([parent, string(column)])
    column_size = Base.summarysize(col_values)
    num_chunks  = ceil(Int, column_size / chunk_size)
    chunks      = Store.chunk_array(col_values, num_chunks)

    println(col_sha1, "; num_chunks: ", num_chunks)
    dataset = Store.ingest_df_column(r, tokenizer, chunks, col_sha1)
    # println(dataset)
    hll = HllSets.HllSet{10}()
    # println(hll)
    dataset = JSON3.write(dataset)
    hll = HllSets.restore(hll, dataset)
    # println(hll)
    entity = Entity.Instance{10}(r, hll)
    println("Column entity instance: ", entity)
end


6f983ba3758e7233f7379a9c7b6ee565808a8de6; num_chunks: 79


LoadError: MethodError: no method matching store_entity(::PyObject, ::Int64, ::HllSet{10}; grad::Float64, op::Nothing, prefix::String)

[0mClosest candidates are:
[0m  store_entity(::PyObject, [91m::String[39m, [91m::String[39m; grad, op, prefix)
[0m[90m   @[39m [35mMain.Entity[39m [90m~/JULIA/SGS/SGS/src/[39m[90m[4msgs_entity.jl:49[24m[39m
[0m  store_entity(::PyObject, [91m::HllSet{P}[39m; grad, op, prefix) where P
[0m[90m   @[39m [35mMain.Entity[39m [90m~/JULIA/SGS/SGS/src/[39m[90m[4msgs_entity.jl:41[24m[39m


In [ ]:
using Random
using Distributions

# Example term frequencies (tf)
terms = ["term1", "term2", "term3", "term4", "term5"]
tf = [10, 20, 30, 25, 15]

# Calculate the total term frequency
total_tf = sum(tf)

# Calculate the probability distribution
probabilities = tf ./ total_tf

# Create a categorical distribution based on the probabilities
term_distribution = Categorical(probabilities)

# Function to sample terms based on the distribution
function sample_terms(terms, term_distribution, num_samples)
    sampled_terms = []
    for _ in 1:num_samples
        term_index = rand(term_distribution)
        push!(sampled_terms, terms[term_index])
    end
    return sampled_terms
end

# Sample 10 terms from the collection
num_samples = 3
sampled_terms = sample_terms(terms, term_distribution, num_samples)

println("Sampled terms: ", sampled_terms)